### CNN卷积
- 本demo使用tf 2.8.0

In [ ]:
#import keras
from tensorflow import keras
import os,shutil
from keras import layers
from keras import models
from keras import optimizers
 
from keras.preprocessing.image import ImageDataGenerator
 
import matplotlib.pyplot as plt
 
keras.__version__

#### 加载数据集

In [ ]:
base_dir = '.\data\smiledata'
train_dir = os.path.join(base_dir, 'train')
 
validation_dir = os.path.join(base_dir, 'validation')
 
train_smile_dir = os.path.join(train_dir, 'smile')
 
train_nosmile_dir = os.path.join(train_dir, 'nosmile')
 
validation_smile_dir = os.path.join(validation_dir, 'smile')
 
validation_nosmile_dir = os.path.join(validation_dir, 'nosmile')
 
print('total training smile images:', len(os.listdir(train_smile_dir)))
print('total training nosmile images:', len(os.listdir(train_nosmile_dir)))
print('total validation smile images:', len(os.listdir(validation_smile_dir)))
print('total validation nosmile images:', len(os.listdir(validation_nosmile_dir)))

#### keras搭建模型

In [ ]:
#当前层核的个数对应本层输出feature个数，当前核的厚度对应上一层feature的个数
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',padding='same',input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu',padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu',padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
 
 
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])
 
model.summary()

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')
 
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')
 
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break
 
 
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50)

#### plot曲线

In [ ]:
hist = history.history
print(hist)
plt.plot(hist['loss'],color='r')
plt.plot(hist['val_loss'],color='g')
plt.plot(hist['binary_accuracy'],color='b')
plt.plot(hist['val_binary_accuracy'],color='k')
plt.title('model loss and acc')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'test_loss','train_acc', 'test_acc'], loc='upper left')
plt.show()

#### 存储h5模型并将h5转换为pb

In [ ]:
model.save('./data/smile_and_nosmile.h5')

In [ ]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

def h5_to_pb(h5_save_path):
    model = tf.keras.models.load_model(h5_save_path, compile=False)
    model.summary()
    full_model = tf.function(lambda Input: model(Input))
    full_model = full_model.get_concrete_function(tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

    # Get frozen ConcreteFunction
    frozen_func = convert_variables_to_constants_v2(full_model)
    frozen_func.graph.as_graph_def()

    layers = [op.name for op in frozen_func.graph.get_operations()]
    print("-" * 50)
    print("Frozen model layers: ")
    for layer in layers:
        print(layer)

    print("-" * 50)
    print("Frozen model inputs: ")
    print(frozen_func.inputs)
    print("Frozen model outputs: ")
    print(frozen_func.outputs)

    # Save frozen graph from frozen ConcreteFunction to hard drive
    tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                      logdir="./pb",
                      name="model.pb",
                      as_text=False)   #可设置.pb存储路径


h5_to_pb('./data/smile_and_nosmile.h5')   #此处填入.h5路径


#### 如何用opencv读取pb模型并预测

In [ ]:
import cv2
import numpy as np
#from cv2.dnn import dnn
 
def get_face_data():
    # 加载Haar级联数据文件，用于检测人面
    face_casecade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    net = cv2.dnn.readNetFromTensorflow('pb/model.pb')
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    camera = cv2.VideoCapture(0)
    count = 1
    while True:
        ret, frame = camera.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_casecade.detectMultiScale(gray,1.3,5)
       # 画出面部位置
        for (x, y, w, h) in faces:
             img = cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
             # 根据检查的位置截取图片并调整截取后的图片大小
             f = cv2.resize(frame[y:y+h, x:x+w], (150, 150))
             
             img_tensor =  cv2.dnn.blobFromImage(f, 1 / 255.0, (150, 150), swapRB=True, crop=False)             
             net.setInput(img_tensor)
             
             ln = net.getUnconnectedOutLayersNames()
             prediction = net.forward(ln)
             
             if prediction[0][0]<0.5:
                result='nosmile'
             else:
                result='smile'
             cv2.putText(frame, result, (x,y), font, 2, (255, 0, 0), 2, cv2.LINE_AA)
#             #cv2.imshow('face',f)
              
             count += 1
             
        cv2.imshow('pic', frame)
        # 停止程序
        if cv2.waitKey(120) & 0xff == ord(' '):
            break
    camera.release()
    cv2.destroyAllWindows()
 
if __name__=='__main__':
    get_face_data()